In [1]:
import json

with open('example_schema.json','r') as f:
    db_schema = json.load(f)
db_schema

{'tables': [{'table_name': 'customers',
   'description': 'Stores information about the e-commerce customers.',
   'columns': {'customer_id': 'Unique identifier for each customer (Primary Key).',
    'first_name': 'First name of the customer.',
    'last_name': 'Last name of the customer.',
    'email': 'Unique email address for the customer.',
    'password_hash': 'Hashed password for account security.',
    'phone_number': 'Contact phone number of the customer.',
    'created_at': 'Timestamp when the customer account was created.'}},
  {'table_name': 'products',
   'description': 'Contains all product information for the store.',
   'columns': {'product_id': 'Unique identifier for each product (Primary Key).',
    'name': 'Name of the product.',
    'description': 'Detailed description of the product.',
    'price': 'Price of the product in a decimal format.',
    'sku': 'Stock Keeping Unit, unique code for the product.',
    'stock_quantity': 'Number of items currently in stock.',
 

In [3]:
from graph import graph
from langchain_core.messages import HumanMessage
from configuration import Configuration

config = Configuration(database_schema=db_schema)

state = graph.invoke(
    {"messages": [HumanMessage("Give me all the users who purchased a product and gave 4+ star reviews. Also give me the products they bought.")]},
    config={"configurable": config.model_dump()}
    )

----Relevance----
The query is relevant to the database schema as it involves customers, products, and reviews. I will replace relative terms with absolute values and rephrase the query for LLM optimization.
Evaluation: PASS
----Router----
Evaluation: PASS
----Generator----
The user wants customers who (1) have purchased products and (2) have left 4+ star reviews, together with the products they have purchased. This means we need to ensure that a customer has at least one order (so they have purchased something) and at least one 4+ star review. For the output, we can list the customer information along with the products they have purchased and/or reviewed. The simplest approach is to select distinct customers who have both orders and reviews with rating >= 4, and include for each such customer the products that they have purchased (via order_items).
Generated SQL: SELECT DISTINCT
    c.customer_id,
    c.first_name,
    c.last_name,
    p.product_id,
    p.name AS product_name
FROM cus

In [4]:
print(state['messages'][-1].content)

```sql
SELECT DISTINCT
    c.customer_id,
    c.first_name,
    c.last_name,
    p.product_id,
    p.name AS product_name
FROM customers c
JOIN orders        o  ON o.customer_id = c.customer_id
JOIN order_items  oi ON oi.order_id = o.order_id
JOIN products     p  ON p.product_id = oi.product_id
JOIN reviews      r  ON r.customer_id = c.customer_id
                    AND r.product_id  = oi.product_id
WHERE r.rating >= 4
ORDER BY c.customer_id, p.product_id;
```
